# 1. Загрузка данных

In [1]:
#!pip install kaggle

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (f1_score, roc_auc_score, 
                            average_precision_score, 
                            balanced_accuracy_score,
                            confusion_matrix)
from sklearn.utils.class_weight import compute_class_weight

from kaggle.api.kaggle_api_extended import KaggleApi


In [3]:
# Инициализация API
api = KaggleApi()
api.authenticate()  # Проверяет ~/.kaggle/kaggle.json

# Скачивание данных соревнования
api.competition_download_files(
    "playground-series-s4e7",  # Название соревнования
    path="./data",             # Папка для сохранения
    quiet=False               # Показывать прогресс
)

# Распаковка архива (если нужно)
import zipfile
with zipfile.ZipFile("./data/playground-series-s4e7.zip", 'r') as zip_ref:
    zip_ref.extractall("./data")

playground-series-s4e7.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

In [5]:
df_train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [6]:
df_test.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,11504798,Female,20,1,47.0,0,< 1 Year,No,2630.0,160.0,228
1,11504799,Male,47,1,28.0,0,1-2 Year,Yes,37483.0,124.0,123
2,11504800,Male,47,1,43.0,0,1-2 Year,Yes,2630.0,26.0,271
3,11504801,Female,22,1,47.0,1,< 1 Year,No,24502.0,152.0,115
4,11504802,Male,51,1,19.0,0,1-2 Year,No,34115.0,124.0,148


Дальше будем работать только с датафреймом df_train, так как df_test не содержат целевую переменную и для обучения не годится

In [7]:
df = df_train

# 2. Обработка данных

### 2.1 Анализ пропущенных значений и дупликатов

In [8]:
missing_analysis = pd.DataFrame({
    'Количество пропусков': df.isnull().sum(),
    'Доля пропусков': df.isnull().mean().round(4)
})
print(missing_analysis[missing_analysis['Количество пропусков'] > 0])

print(f"Полных дубликатов строк: {df.duplicated().sum()}")
print(f"Частичных дубликатов по ID: {df['id'].duplicated().sum()}")


Empty DataFrame
Columns: [Количество пропусков, Доля пропусков]
Index: []
Полных дубликатов строк: 0
Частичных дубликатов по ID: 0


id в обучении не пригодится

In [9]:
df.drop('id', axis=1, inplace=True)

### 2.2 Сбалансированность классов

Посмотрим на соотношение классов Response. Это поможет нам потом выбрать правильные метрики

In [10]:
class_dist = df['Response'].value_counts(normalize=True)

print(f"Распределение классов:\n{class_dist}\n")
print(f"Соотношение классов: {class_dist[0]/class_dist[1]:.1f}:1")

Распределение классов:
Response
0    0.877003
1    0.122997
Name: proportion, dtype: float64

Соотношение классов: 7.1:1


Видно, что классы несбалансированы. Поэтому нельзя использовать в качестве метрики качества accuracy, например. Также, для обучения моделей нужно компенсировать малый класс через больший вес ошибки на этапе обучение. Для этого используем compute_class_weight

In [11]:
# Разделение с сохранением распределения
X = df.drop('Response', axis=1)
y = df['Response']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y)

# Вычисление весов классов
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
print(f"\nВеса классов для компенсации дисбаланса: {class_weight_dict}")


Веса классов для компенсации дисбаланса: {0: 0.570123648963296, 1: 4.065131074836882}


Категориальные и числовые переменные

In [12]:
num_cols = ['Age', 'Driving_License', 'Region_Code', 'Previously_Insured',
            'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']
cat_cols = ['Gender', 'Vehicle_Age', 'Vehicle_Damage']
all_cols = num_cols + cat_cols

Сохраним датасет с числовыми признаками для бэйзлайн модели

In [13]:
X_train_base = X_train[num_cols]
X_test_base = X_test[num_cols]

### 2.3 Обработка категориальных признаков и нормализация

Сохраним датасет для последующей обработки категориальных признаков и нормализации

In [14]:
X_train_proc = X_train.copy()
X_test_proc = X_test.copy()

In [15]:
le = LabelEncoder()
for col in cat_cols:
    X_train_proc[col] = le.fit_transform(X_train_proc[col])
    X_test_proc[col] = le.transform(X_test_proc[col])

In [16]:
X_train_proc.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
4759874,0,25,1,50.0,1,1,0,35283.0,152.0,65
11227509,0,52,1,28.0,0,0,1,47269.0,26.0,114
700827,1,44,1,36.0,1,0,0,31854.0,124.0,34
10859621,0,49,1,50.0,0,0,1,23086.0,160.0,297
11120384,0,22,1,37.0,1,1,0,25416.0,152.0,193


Нормализация

In [17]:
scaler = MinMaxScaler()
X_train_proc[all_cols] = scaler.fit_transform(X_train_proc[all_cols])
X_test_proc[all_cols] = scaler.transform(X_test_proc[all_cols])


# 3. Генерация признаков и отбор

Признаки на основе полиномиальных комбинаций числовых признаков

In [18]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_proc[all_cols])
X_test_poly = poly.transform(X_test_proc[all_cols])

Применяем SelectKBest для отбора 10 лучших признаков

In [19]:
selector = SelectKBest(f_classif, k=10)
X_train_sel = selector.fit_transform(X_train_poly, y_train)
X_test_sel = selector.transform(X_test_poly)

# 4. Обучение моделей на сырых и разной степени обработки данных. Сравнение результатов

Создадим функцию для оценки модели по заданным метрикам

In [20]:
def evaluate_imbalanced(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return {
        'F1-score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_proba),
        'PR-AUC': average_precision_score(y_test, y_proba),
        'Balanced Accuracy': balanced_accuracy_score(y_test, y_pred),
        'Sensitivity': tp / (tp + fn),
        'Specificity': tn / (tn + fp),
        'G-Mean': np.sqrt((tp/(tp+fn)) * (tn/(tn+fp)))
    }

Краткое описание выбранных метрик и причин их использования для задачи бинарной классификации с дисбалансом классов:

1. **F1-score** (Гармоническое среднее precision и recall)  
   - Почему: Балансирует между точностью и полнотой, что критично при дисбалансе классов  
   - Формула: `F1 = 2*(Precision*Recall)/(Precision+Recall)`

2. **PR-AUC** (Precision-Recall AUC)  
   - Почему: Лучше ROC-AUC для сильно дисбалансированных данных, фокусируется на качестве предсказаний positive-класса  
   - Особенность: Оценивает кривую precision-recall вместо ROC-кривой

3. **Balanced Accuracy**  
   - Почему: Учитывает accuracy для каждого класса отдельно, затем усредняет  
   - Формула: `(Sensitivity + Specificity)/2`

4. **G-Mean** (Геометрическое среднее sensitivity и specificity)  
   - Почему: Наказывает за большие различия в ошибках между классами  
   - Формула: `√(Sensitivity * Specificity)`

**Дополнительно в коде используются:**
- **Sensitivity (Recall)** - способность находить все positive-случаи  
- **Specificity** - способность правильно идентифицировать negative-случаи

**Почему не accuracy?**  
При дисбалансе 80:20 модель может получить accuracy=0.8, просто всегда предсказывая majority-класс, что бесполезно на практике. Выбранные метрики устойчивы к дисбалансу.

In [21]:
results = {}

### 4.1 Бейзлайн модель

In [22]:
lr_base = LogisticRegression(class_weight=class_weight_dict, max_iter=1000)
results['Baseline'] = evaluate_imbalanced(lr_base, X_train_base, X_test_base, y_train, y_test)

### 4.2 С обработкой категориальных и нормализацией

In [23]:
lr_proc = LogisticRegression(class_weight=class_weight_dict, max_iter=1000)
results['Processed'] = evaluate_imbalanced(lr_proc, X_train_proc[all_cols], X_test_proc[all_cols], y_train, y_test)

### 4.3 С полиномиальными признаками

In [24]:
lr_poly = LogisticRegression(class_weight=class_weight_dict, max_iter=1000)
results['Polynomial'] = evaluate_imbalanced(lr_poly, X_train_poly, X_test_poly, y_train, y_test)

Обучение занимает много времени. Можно будет попробовать снизить размерность через PCA

### 4.4 С отбором признаков

In [25]:
lr_sel = LogisticRegression(class_weight=class_weight_dict, max_iter=1000)
results['Selected Features'] = evaluate_imbalanced(lr_sel, X_train_sel, X_test_sel, y_train, y_test)


### 4.5 Табличка метриками разных мелей

Сравнение моделей по ключевым метрикам

In [26]:
results_df = pd.DataFrame(results).T.round(3)

key_metrics = ['F1-score', 'PR-AUC', 'Balanced Accuracy', 'G-Mean']
results_df[key_metrics]


,F1-score,PR-AUC,Balanced Accuracy,G-Mean
Baseline,0.263,0.165,0.590,0.588
Processed,0.402,0.299,0.787,0.763
Polynomial,0.432,0.346,0.800,0.788
Selected Features,0.402,0.257,0.787,0.763


Лучшие метрики у модели со сгенерированными полиномиальными признаками. Модель с обработкой категориальных признаков и нормализацией тоже показывает хорошие результаты, но она существенно быстрее

Все посчитанные метрики

In [27]:
results_df

,F1-score,ROC-AUC,PR-AUC,Balanced Accuracy,Sensitivity,Specificity,G-Mean
Baseline,0.263,0.660,0.165,0.590,0.540,0.640,0.588
Processed,0.402,0.824,0.299,0.787,0.981,0.593,0.763
Polynomial,0.432,0.849,0.346,0.800,0.934,0.665,0.788
Selected Features,0.402,0.804,0.257,0.787,0.981,0.593,0.763


Если будет время можно будет добавить метрику скорости работы модели, а также понизить размерность модели с полиномиальными признаками через метод главных компонент